## Basic GPT 
Implementation of GPT from scratch in Tensorflow
### Imports

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Configuration

In [2]:
EMBEDDING_DIM = 384 # the dimension of the word embeddings
N_HEADS = 6 #  the number of attention heads
NUM_BLOCKS = 6 # Number of transformer blocks 12 in gpt2, 3 in gpt nano
VALIDATION_SPLIT = 0.2 # the fraction of data to be used for validation
SEED = 42 # the random seed for reproducibility
SEQ_LEN = 256 # 512  # Length of training sequences, in tokens

In [3]:
BATCH_SIZE=128
DROPOUT = 0.2

In [4]:
MAX_ITERATIONS = 5000
EVAL_INTERVAL = 500
LEARNING_RATE = 3e-4
EVAL_ITERATIONS = 200

In [5]:
# Set random seed
tf.random.set_seed(SEED)

## Load and pre-process data
Ingest a text file

In [6]:
#Loading data
with open('/kaggle/input/coriolanus/coriolanus.txt', 'r', encoding='utf-8') as f:
    text = f.read()

### Create a dictionary
A simple, character-based dictionary and tokenization scheme

In [7]:
# Create character mappings
chars = sorted(list(set(text)))
vocab_size = len(chars)
print (vocab_size)
chars[:30]

65


['\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q']

### Tokenize and de-tokenize

In [8]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [9]:
stoi

{'\n': 0,
 ' ': 1,
 '!': 2,
 '$': 3,
 '&': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '3': 9,
 ':': 10,
 ';': 11,
 '?': 12,
 'A': 13,
 'B': 14,
 'C': 15,
 'D': 16,
 'E': 17,
 'F': 18,
 'G': 19,
 'H': 20,
 'I': 21,
 'J': 22,
 'K': 23,
 'L': 24,
 'M': 25,
 'N': 26,
 'O': 27,
 'P': 28,
 'Q': 29,
 'R': 30,
 'S': 31,
 'T': 32,
 'U': 33,
 'V': 34,
 'W': 35,
 'X': 36,
 'Y': 37,
 'Z': 38,
 'a': 39,
 'b': 40,
 'c': 41,
 'd': 42,
 'e': 43,
 'f': 44,
 'g': 45,
 'h': 46,
 'i': 47,
 'j': 48,
 'k': 49,
 'l': 50,
 'm': 51,
 'n': 52,
 'o': 53,
 'p': 54,
 'q': 55,
 'r': 56,
 's': 57,
 't': 58,
 'u': 59,
 'v': 60,
 'w': 61,
 'x': 62,
 'y': 63,
 'z': 64}

### Tokenize the text and generate training / validation data sets

In [10]:
# Train and test splits
data = tf.constant(encode(text), dtype=tf.int64)
n = int((1.0-VALIDATION_SPLIT) * len(data))
train_data = data[:n]
val_data = data[n:]

In [11]:
val_data[:100]

<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([63, 53, 59,  1, 39, 56, 43,  6,  0, 32, 46, 39, 58,  1, 47, 57,  6,
        1, 39,  1, 61, 53, 51, 39, 52, 11,  1, 47, 44,  1, 63, 53, 59,  1,
       40, 43,  1, 51, 53, 56, 43,  6,  1, 63, 53, 59,  5, 56, 43,  1, 52,
       53, 52, 43, 11,  0, 21, 44,  1, 63, 53, 59,  1, 40, 43,  1, 53, 52,
       43,  6,  1, 39, 57,  1, 63, 53, 59,  1, 39, 56, 43,  1, 61, 43, 50,
       50,  1, 43, 62, 54, 56, 43, 57, 57,  5, 42,  0, 14, 63,  1])>

## Define the model

In [12]:
# Data loading into batches
# the "label" y is the next token
def get_batch(split):
    data_split = train_data if split == 'train' else val_data
    ix = tf.random.uniform(shape=(BATCH_SIZE,), maxval=len(data_split) - SEQ_LEN, dtype=tf.int32)
    x = tf.stack([data_split[i:i+SEQ_LEN] for i in ix])
    y = tf.stack([data_split[i+1:i+SEQ_LEN+1] for i in ix])
    return x, y

In [13]:
# Calculating loss of the model
def estimate_loss(model):
    out = {}
    model.trainable = False
    for split in ['train', 'val']:
        losses = tf.TensorArray(tf.float32, size=EVAL_ITERATIONS)
        for k in range(EVAL_ITERATIONS):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses = losses.write(k, loss)
        out[split] = losses.stack().numpy().mean()
    model.trainable = True
    return out

In [14]:
class Head(tf.keras.layers.Layer):
    """ one head of self-attention """

    def __init__(self, head_size):
        super(Head, self).__init__()
        self.key = tf.keras.layers.Dense(head_size, use_bias=False)
        self.query = tf.keras.layers.Dense(head_size, use_bias=False)
        self.value = tf.keras.layers.Dense(head_size, use_bias=False)

        tril = tf.linalg.band_part(tf.ones((SEQ_LEN, SEQ_LEN)), -1, 0)
        self.tril = tf.constant(tril)

        self.dropout = tf.keras.layers.Dropout(DROPOUT)

    def call(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B, T, C = x.shape
        k = self.key(x)   # (B, T, hs)
        q = self.query(x) # (B, T, hs)

        # compute attention scores 
        wei = tf.matmul(q, tf.transpose(k, perm=[0, 2, 1])) * tf.math.rsqrt(tf.cast(k.shape[-1], tf.float32))  # (B, T, T)
        wei = tf.where(self.tril[:T, :T] == 0, float('-inf'), wei)  # (B, T, T)
        wei = tf.nn.softmax(wei, axis=-1)  # (B, T, T)
        wei = self.dropout(wei)

        # perform the weighted aggregation of the values
        v = self.value(x)  # (B, T, hs)
        out = tf.matmul(wei, v)  # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

In [15]:
class MultiHeadAttention(tf.keras.layers.Layer):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super(MultiHeadAttention, self).__init__()
        self.heads = [Head(head_size) for _ in range(num_heads)]
        self.proj = tf.keras.layers.Dense(EMBEDDING_DIM)
        self.dropout = tf.keras.layers.Dropout(DROPOUT)

    def call(self, x):
        out = tf.concat([h(x) for h in self.heads], axis=-1)
        out = self.dropout(self.proj(out))
        return out

In [16]:
class FeedForward(tf.keras.layers.Layer):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super(FeedForward, self).__init__()
        self.net = tf.keras.Sequential([
            tf.keras.layers.Dense(4 * n_embd),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Dense(n_embd),
            tf.keras.layers.Dropout(DROPOUT),
        ])

    def call(self, x):
        return self.net(x)

In [17]:
class Block(tf.keras.layers.Layer):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        super(Block, self).__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.ln2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    def call(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [18]:
class GPTLanguageModel(tf.keras.Model):

    def __init__(self):
        super(GPTLanguageModel, self).__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = tf.keras.layers.Embedding(vocab_size, EMBEDDING_DIM)
        self.position_embedding_table = tf.keras.layers.Embedding(SEQ_LEN, EMBEDDING_DIM)
        self.blocks = tf.keras.Sequential([Block(EMBEDDING_DIM, n_head=N_HEADS) for _ in range(NUM_BLOCKS)])
        self.ln_f = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.lm_head = tf.keras.layers.Dense(vocab_size, kernel_initializer='normal', bias_initializer='zeros')

    def call(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx)  # (B,T,C)
        pos_emb = self.position_embedding_table(tf.range(T, dtype=tf.float32))  # (T,C)
        x = tok_emb + pos_emb  # (B,T,C)
        x = self.blocks(x)  # (B,T,C)
        x = self.ln_f(x)  # (B,T,C)
        logits = self.lm_head(x)  # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = tf.reshape(logits, (B * T, C))
            targets = tf.reshape(targets, (B * T,))
            loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)(targets, logits)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last SEQ_LEN tokens
            idx_cond = idx[:, -SEQ_LEN:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = tf.nn.softmax(logits, axis=-1)  # (B, C)
            # sample from the distribution
            idx_next = tf.random.categorical(tf.math.log(probs), num_samples=1, dtype=tf.int64)  # (B, 1)
            # append sampled index to the running sequence
            idx = tf.concat([idx, idx_next], axis=1)  # (B, T+1)
        return idx


In [19]:
# Initializing model with pre-trained weights. Use this if you don't want to re-train the model.
#model = GPTLanguageModel()
#dummy_input = tf.constant([[0]], dtype=tf.int32)  # Example input, adjust shape as needed
#model(dummy_input)
#model.load_weights('/kaggle/working/gpt_model_basic.h5')

## Train the model

In [ ]:
#Training the model. GPU is recommended for training.

model = GPTLanguageModel()
optimizer = tf.keras.optimizers.AdamW(LEARNING_RATE)

for iter in range(MAX_ITERATIONS):

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    with tf.GradientTape() as tape:
        logits, loss = model(xb, yb)

    # every once in a while evaluate the loss on train and val sets 
    # MUST happen after tape update
    if iter % EVAL_INTERVAL == 0 or iter == MAX_ITERATIONS - 1:
        losses = estimate_loss(model)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))


## Generation 
with pre-trained model

In [ ]:
# generate from the model
context = tf.zeros((1, 1), dtype=tf.int64)
generated_sequence = model.generate(context, max_new_tokens=500).numpy()
print(decode(generated_sequence[0]))

## Save model

In [ ]:
model.save_weights('/kaggle/working/gpt_model_basic.h5')